In [1]:
import os
import pandas as pd

from dotenv import load_dotenv
from azure.storage.blob import BlobServiceClient, ContainerClient
from tqdm import tqdm

In [2]:
load_dotenv()

account_url = os.getenv('SAME_AZURE_ACCOUNT_URL', 'https://your-account.blob.core.windows.net')
account_name = os.getenv('SAME_AZURE_ACCOUNT_NAME', 'your-account-name')
container = os.getenv('SAME_AZURE_CONTAINER', 'your-container-name')
sas_token = os.getenv('SAME_AZURE_SAS_TOKEN', 'your-sas-token-here')

In [3]:
container_url = f"https://{account_name}.blob.core.windows.net/{container}?{sas_token}"
container_client = ContainerClient.from_container_url(container_url)

In [4]:
trimmed_videos = pd.read_csv('./../../data/videos-trimmed.csv')

In [5]:
print(len(trimmed_videos))
trimmed_videos.head()

2277


,video_base,t_start_sec_x,t_end_sec_x,trimmed_name,start_frame12,window_activity,lat,long,timestamp,id,id_person,camera
0,columpios_cam4-2024-11-21 18:53:10.mp4,13.833333,17.833333,trimmed_columpios_cam4-2024-11-21_18_53_10_00-...,166.0,7.713624,25.653061,-100.285843,2024-11-22 00:53:10.041613+00:00,columpios_cam4-2024-11-21_185310.mp4:17527661-...,17527661-d095-4e94-bad5-cf142e0d4a79,columpios_cam4
1,columpioscam3-2024-09-25 11:41:35.mp4,6.250000,10.250000,trimmed_columpioscam3-2024-09-25_11_41_35_00-0...,75.0,7.713624,25.653034,-100.286041,2024-09-25 17:41:35.045100+00:00,columpioscam3-2024-09-25_114135.mp4:96:0,96,columpioscam3
2,columpioscam3-2024-10-03 18:29:30.mp4,12.916667,16.916667,trimmed_columpioscam3-2024-10-03_18_29_30_00-0...,155.0,7.713624,25.652994,-100.286118,2024-10-04 00:29:30.051348+00:00,columpioscam3-2024-10-03_182930.mp4:66a106a0-b...,66a106a0-bd03-49e9-8418-8c6ae98d020c,columpioscam3
3,columpioscam1-2024-11-17 10:25:24.mp4,10.000000,14.000000,trimmed_columpioscam1-2024-11-17_10_25_24_00-0...,120.0,7.713624,25.653151,-100.285637,2024-11-17 16:25:24.041494+00:00,columpioscam1-2024-11-17_102524.mp4:85745e56-a...,85745e56-a3e9-4b62-bb2d-c1e8ee152f86,columpioscam1
4,columpios_cam4-2024-11-21 19:16:11.mp4,6.333333,10.333333,trimmed_columpios_cam4-2024-11-21_19_16_11_00-...,76.0,7.713624,25.653048,-100.285912,2024-11-22 01:16:11.041568+00:00,columpios_cam4-2024-11-21_191611.mp4:addad6d5-...,addad6d5-1fe6-484f-964d-3302e3899325,columpios_cam4


In [6]:
temp_dir = "./../../data/temp/videos/trimmed"
os.makedirs(temp_dir, exist_ok=True)

In [7]:
def download_videos(videos_df):
    for _, row in tqdm(videos_df.iterrows(), total=len(videos_df), desc="Downloading videos"):
        blob_name = row["trimmed_name"]
        local_path = os.path.join(temp_dir, blob_name)

        if os.path.exists(local_path):
            print(f"[INFO] Skipping {blob_name}, already exists.")
            continue

        try:
            blob_client = container_client.get_blob_client(blob_name)
            with open(local_path, "wb") as f:
                download_stream = blob_client.download_blob()
                f.write(download_stream.readall())
            print(f"Downloaded: {blob_name}")
        except Exception as e:
            print(f"⚠️ Error downloading {blob_name}: {e}")

    print("\nAll downloads complete.")

In [ ]:
download_videos(trimmed_videos)